# Lab 2 - Travelling Salesman Problem (TSP)

In [197]:
import pandas as pd
import numpy as np

In [198]:
PATH = "cities/"
INSTANCE = "vanuatu.csv"

In [199]:
cities = pd.read_csv(f"{PATH}{INSTANCE}", header=None, names=["City", "x", "y"])
cities, type(cities)

(         City      x       y
 0     Isangel -19.53  169.28
 1    Lakatoro -16.09  167.40
 2     Longana -15.30  168.00
 3  Luganville -15.51  167.15
 4      Norsup -16.07  167.39
 5   Port Olry -15.05  167.05
 6        Sola -13.87  167.55
 7        Vila -17.74  168.31,
 pandas.core.frame.DataFrame)

In [200]:
coordinates = cities[["x", "y"]].to_numpy()
coordinates

array([[-19.53, 169.28],
       [-16.09, 167.4 ],
       [-15.3 , 168.  ],
       [-15.51, 167.15],
       [-16.07, 167.39],
       [-15.05, 167.05],
       [-13.87, 167.55],
       [-17.74, 168.31]])

## Helper functions

In [201]:
def distance(start, end):
    return ((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2) ** 0.5


def distance_matrix(coordinates: np.ndarray) -> np.ndarray:
    num_cities = coordinates.shape[0]
    dist_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(i+1):
            d = distance(coordinates[i], coordinates[j])
            dist_matrix[i, j] = d
            dist_matrix[j, i] = d
            if i == j:
                dist_matrix[i, j] = 0
            
    return dist_matrix


def cost(solution: np.ndarray, dist_matrix: np.ndarray) -> np.float64 | float:
    """Cost of a cycle"""
    return np.sum(
        np.array([
            dist_matrix[solution[i]][solution[i+1]] for i in range(len(solution)-1)
        ]
    )) + dist_matrix[solution[-1]][solution[0]]

## Greedy

In [202]:
def greedy_solve(coordinates: np.ndarray):
    """Greedy algorithm with random initialization: sub-optimal"""
    dist_matrix = distance_matrix(coordinates)
    temp = dist_matrix.copy()
    
    num_cities = coordinates.shape[0]
    visited = np.full(num_cities, False)
    start_index = np.random.randint(0, num_cities)
    
    solution = -np.ones(num_cities+1, dtype=np.int16)
    solution[0], visited[0] = start_index, True
    for step in range(num_cities-1):
        temp[:, start_index] = np.inf
        next_index = np.argmin(temp[start_index])
        
        solution[step+1] = next_index
        start_index = next_index
        visited[start_index] = True
        
    solution[-1] = solution[0]
    
    return solution, cost(solution, dist_matrix), dist_matrix

In [203]:
solution, cost_sol, matrix = greedy_solve(coordinates)
for (start, end) in zip(solution[:-1], solution[1:]):
    print(f"{start} -> {end}", matrix[start, end])
    
print((solution, cost_sol))
print(matrix)

1 -> 4 0.022360679775006158
4 -> 3 0.6092618484691055
3 -> 5 0.4707440918375907
5 -> 2 0.982344135219414
2 -> 6 1.499133082818198
6 -> 7 3.9439193703725715
7 -> 0 2.035927307150235
0 -> 1 3.920204076320516
(array([1, 4, 3, 5, 2, 6, 7, 0, 1], dtype=int16), np.float64(13.483894591962638))
[[0.         3.92020408 4.41942304 4.54942854 3.94254994 5.00432813
  5.91848798 2.03592731]
 [3.92020408 0.         0.99201814 0.63158531 0.02236068 1.0973149
  2.2250618  1.88430358]
 [4.41942304 0.99201814 0.         0.87555697 0.98234414 0.98234414
  1.49913308 2.45961379]
 [4.54942854 0.63158531 0.87555697 0.         0.60926185 0.47074409
  1.68807583 2.51366267]
 [3.94254994 0.02236068 0.98234414 0.60926185 0.         1.0751744
  2.20581051 1.90664627]
 [5.00432813 1.0973149  0.98234414 0.47074409 1.0751744  0.
  1.28156155 2.97047134]
 [5.91848798 2.2250618  1.49913308 1.68807583 2.20581051 1.28156155
  0.         3.94391937]
 [2.03592731 1.88430358 2.45961379 2.51366267 1.90664627 2.97047134
  3